> Please go through Giba's post and kernel  to underrstand what this leak is all about
> https://www.kaggle.com/titericz/the-property-by-giba (kernel)
> https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61329 (post)
> 
> Also, go through this Jiazhen's kernel which finds more columns to exploit leak
> https://www.kaggle.com/johnfarrell/giba-s-property-extended-result
> 
> I just exploit data property in brute force way and then fill in remaining by row non zero means! This should bring everyone on level-playing field.
> 
> **Let the competition begin! :D**

### Just some small modifications from [original baseline](https://www.kaggle.com/tezdhar/breaking-lb-fresh-start)~
- The leak rows are calculated separately on train/test set
- Calculated the leaky values, correctness, for each lag
- Hope this can help to do some *lag_selection*

### Update leak process codes to Dmitry Frumkin's *fast* [version](https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2)
- The result of Dmitry's original function and result of Hasan's function seem slightly different
- Modified to make the output consistent with Hasan's function (Seems better score)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# print(os.listdir("../input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

# Any results you write to the current directory are saved as output.

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [3]:
test["target"] = train["target"].mean()

# Determine Extra Features to use

In [222]:
rows = [1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245, 1383, 2071, 3492, 378, 2971, 2366, 4414, 2790, 3979, 193, 1189, 3516, 810, 4443, 3697, 235, 1382, 4384, 3418, 4396, 921, 3176, 650]

def element_wise_comparison(i1, i2, thresh=3):
    i1 = np.array(i1)
    i2 = np.array(i2)
    dif = i1 - i2
    count = np.count_nonzero(dif)
    if count <= thresh:
        return True
    else:
        return False

def solve_ties_in_nb_of_fillers(df, counts, stop_thresh=2):
    args = np.argsort(counts)
    stop = np.squeeze(np.where(counts[args] == len(df.index)-stop_thresh))
#     print(stop, type(stop)
    try:
        stop = int(stop)
    except:
          return [None]
#     print(args)
    u, c = np.unique(counts[args[:stop][::-1]], return_counts=True)
    conflicts = u[np.where(c > 1)]
    print(conflicts)
    for c in conflicts:
        print(c)
        idx = np.squeeze(np.where(counts == c))
        ref_id = np.squeeze(np.where(counts == c+1))
#         print('ref_id', ref_id)
        ref = df.iloc[:-1, int(ref_id)]
#         print(ref)
#         print(df.iloc[1:, idx[0]].values - ref.values)
#         print(df.iloc[1:, idx[1]].values - ref.values)
        dif1 = np.count_nonzero(df.iloc[1:, idx[0]].values - ref.values)
        dif2 = np.count_nonzero(df.iloc[1:, idx[1]].values - ref.values)
        print('difs', dif1, dif2)
        print('idx_value0', df.iloc[:, idx[0]], df.iloc[:, idx[1]])
        loc0  = np.where(args == idx[0])
        loc1  = np.where(args == idx[1])
        print(loc0, loc1)

        if dif2 > dif1:
            args[loc0], args[loc1] = args[loc1], args[loc0] 

#     print(args)
    return args[:stop][::-1]
    
        
def find_filler_value(df, idx):
    u, c = np.unique(df.iloc[0,:].values, return_counts=True)
    arg = np.argsort(c)
    return u[arg][-idx]

def find_col_order_by_filler(df, filler, stop_thresh=2, nb=40):
    counts = []
    for c in df.columns:
        v = df[c].values - filler
        counts.append(np.count_nonzero(v))
    counts = np.array(counts)
    args = solve_ties_in_nb_of_fillers(df, counts, stop_thresh)
    if not all(args):
        return [None]
    return df.iloc[:, args]

def find_leading_col(raw_df, sorted_df):
    curr_lead = sorted_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'}).iloc[0, 0][1:]
    cols_to_match = raw_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'})
    for c in cols_to_match.index:
        if element_wise_comparison(cols_to_match.loc[c, 'key'][:-1], curr_lead):
            print('Leading col found: {}'.format(c))
            return pd.concat([raw_df[c], sorted_df], axis=1)
        
    print('No lead found')
    return sorted_df
    
def find_n_cols_groups(df, n_groups=40, stop_thresh=2):
    col_groups = []
    for i in range(n_groups):
        filler = find_filler_value(tmp, i)
        df_view = find_col_order_by_filler(tmp, filler)
        if not all(df_view):
            continue
        for _ in range(stop_thresh + 1):
            df_view = find_leading_col(tmp, df_view)
        col_groups.append(list(df_view.columns))
    return col_groups

In [242]:
extra_feats

[['ced6a7e91',
  '9df4daa99',
  '83c3779bf',
  'edc84139a',
  'f1e0ada11',
  '73687e512',
  'aa164b93b',
  '342e7eb03',
  'cd24eae8a',
  '8f3740670',
  '2b2a10857',
  'a00adf70e',
  '3a48a2cd2',
  'a396ceeb9',
  '9280f3d04',
  'fec5eaf1a',
  '5b943716b',
  '22ed6dba3',
  '5547d6e11',
  'e222309b0',
  '5d3b81ef8',
  '1184df5c2',
  '2288333b4',
  'f39074b55',
  'a8b721722',
  '13ee58af1',
  'fb387ea33',
  '4da206d28',
  'ea4046b8d',
  'ef30f6be5',
  'b85fa8b27',
  '2155f5e16',
  '794e93ca6',
  '070f95c99',
  '939f628a7',
  '7e814a30d',
  'a6e871369',
  '0dc4d6c7d',
  'bc70cbc26',
  'aca228668']]

In [228]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426',
        '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]
    
extra_feats = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11','73687e512',
               'aa164b93b','342e7eb03','cd24eae8a','8f3740670','2b2a10857','a00adf70e',
               '3a48a2cd2','a396ceeb9','9280f3d04','fec5eaf1a','5b943716b','22ed6dba3',
               '5547d6e11','e222309b0','5d3b81ef8','1184df5c2','2288333b4','f39074b55',
               'a8b721722','13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
               'b85fa8b27','2155f5e16','794e93ca6','070f95c99','939f628a7','7e814a30d',
               'a6e871369','0dc4d6c7d','bc70cbc26','aca228668']

# extra_feats = [[]]
tmp = train.loc[rows, :].drop('ID', axis=1)
extra_feats = find_n_cols_groups(tmp, 30)
extra_feats

[ 6 23 29]
6
difs 0 0
idx_value0 1757    662000
3809    662000
511     662000
3798    662000
625     662000
3303    662000
4095    662000
1283    662000
4209    662000
1696    662000
3511    662000
816     662000
245     662000
1383    662000
2071    662000
3492    662000
378     662000
2971    662000
2366    662000
4414    662000
2790    662000
3979    662000
193     662000
1189    662000
3516    662000
810     662000
4443    662000
3697    662000
235     662000
1382    662000
4384         0
3418         0
4396         0
921          0
3176         0
650          0
Name: 947c7c3e8, dtype: int64 1757    662000
3809    662000
511     662000
3798    662000
625     662000
3303    662000
4095    662000
1283    662000
4209    662000
1696    662000
3511    662000
816     662000
245     662000
1383    662000
2071    662000
3492    662000
378     662000
2971    662000
2366    662000
4414    662000
2790    662000
3979    662000
193     662000
1189    662000
3516    662000
810     662000
4443   

Leading col found: 543c24e33
Leading col found: 18562fc62
Leading col found: 81de0d45e
[ 6 23 29]
6
difs 0 1
idx_value0 1757    3000000
3809    3000000
511     3000000
3798    3000000
625     3000000
3303    3000000
4095    3000000
1283    3000000
4209    3000000
1696    3000000
3511    3000000
816     3000000
245     3000000
1383    3000000
2071    3000000
3492    3000000
378     3000000
2971    3000000
2366    3000000
4414    3000000
2790    3000000
3979    3000000
193     3000000
1189    3000000
3516    3000000
810     3000000
4443    3000000
3697    3000000
235     3000000
1382    3000000
4384     174000
3418          0
4396          0
921           0
3176          0
650           0
Name: 380d54fad, dtype: int64 1757    3000000
3809    3000000
511     3000000
3798    3000000
625     3000000
3303    3000000
4095    3000000
1283    3000000
4209    3000000
1696    3000000
3511    3000000
816     3000000
245     3000000
1383    3000000
2071    3000000
3492    3000000
378     3000000
29

Leading col found: 9126049d8
Leading col found: a3e023f65
[ 6 23 29]
6
difs 2 1
idx_value0 1757     2865694.28
3809     2865694.28
511      2865694.28
3798     2865694.28
625      2865694.28
3303     2865694.28
4095     2865694.28
1283     2865694.28
4209     2865694.28
1696     2865694.28
3511     2865694.28
816      2865694.28
245      2865694.28
1383     2865694.28
2071     2865694.28
3492     2865694.28
378      2865694.28
2971     2865694.28
2366     2865694.28
4414     2865694.28
2790     2865694.28
3979     2865694.28
193      2865694.28
1189     2865694.28
3516     2865694.28
810      2865694.28
4443     2865694.28
3697     2865694.28
235      2865694.28
1382     2865694.28
4384    10484285.72
3418           0.00
4396           0.00
921            0.00
3176           0.00
650       500000.00
Name: 8367dfc36, dtype: float64 1757    2865694.28
3809    2865694.28
511     2865694.28
3798    2865694.28
625     2865694.28
3303    2865694.28
4095    2865694.28
1283    2865694.28
4209 

Leading col found: 2b8851e90
Leading col found: 48b839509
[ 6 23 29]
6
difs 0 2
idx_value0 1757     2400000
3809     2400000
511      2400000
3798     2400000
625      2400000
3303     2400000
4095     2400000
1283     2400000
4209     2400000
1696     2400000
3511     2400000
816      2400000
245      2400000
1383     2400000
2071     2400000
3492     2400000
378      2400000
2971     2400000
2366     2400000
4414     2400000
2790     2400000
3979     2400000
193      2400000
1189     2400000
3516     2400000
810      2400000
4443     2400000
3697     2400000
235      2400000
1382     2400000
4384           0
3418           0
4396           0
921     50000000
3176           0
650            0
Name: cacc1edae, dtype: int64 1757     2400000
3809     2400000
511      2400000
3798     2400000
625      2400000
3303     2400000
4095     2400000
1283     2400000
4209     2400000
1696     2400000
3511     2400000
816      2400000
245      2400000
1383     2400000
2071     2400000
3492     240

Leading col found: 93efdb50f
Leading col found: 920a04ee2
[ 6 23 29]
6
difs 5 1
idx_value0 1757    2202500.00
3809    2202500.00
511     2202500.00
3798    2202500.00
625     2202500.00
3303    2202500.00
4095    2202500.00
1283    2202500.00
4209    2202500.00
1696    2202500.00
3511    2202500.00
816     2202500.00
245     2202500.00
1383    2202500.00
2071    2202500.00
3492    2202500.00
378     2202500.00
2971    2202500.00
2366    2202500.00
4414    2202500.00
2790    2202500.00
3979    2202500.00
193     2202500.00
1189    2202500.00
3516    2202500.00
810     2202500.00
4443    2202500.00
3697    2202500.00
235     2202500.00
1382    2202500.00
4384    7255906.66
3418          0.00
4396    2580000.00
921           0.00
3176    1955000.00
650      700000.00
Name: b612f9b7e, dtype: float64 1757    2202500.0
3809    2202500.0
511     2202500.0
3798    2202500.0
625     2202500.0
3303    2202500.0
4095    2202500.0
1283    2202500.0
4209    2202500.0
1696    2202500.0
3511    22025

...

Leading col found: 2e103d632
Leading col found: 63c094ba4
Leading col found: 87ffda550
[ 6 23 29]
6
difs 5 1
idx_value0 1757    3255483.88
3809    3255483.88
511     3255483.88
3798    3255483.88
625     3255483.88
3303    3255483.88
4095    3255483.88
1283    3255483.88
4209    3255483.88
1696    3255483.88
3511    3255483.88
816     3255483.88
245     3255483.88
1383    3255483.88
2071    3255483.88
3492    3255483.88
378     3255483.88
2971    3255483.88
2366    3255483.88
4414    3255483.88
2790    3255483.88
3979    3255483.88
193     3255483.88
1189    3255483.88
3516    3255483.88
810     3255483.88
4443    3255483.88
3697    3255483.88
235     3255483.88
1382    3255483.88
4384    3225923.08
3418    1549200.00
4396    3795142.86
921      714888.88
3176    3889714.28
650     4547500.00
Name: 47cd6e6e4, dtype: float64 1757    3255483.88
3809    3255483.88
511     3255483.88
3798    3255483.88
625     3255483.88
3303    3255483.88
4095    3255483.88
1283    3255483.88
4209    3255

Leading col found: c0004231c
Leading col found: 48df886f9
[ 6 23 29]
6
difs 1 5
idx_value0 1757    1563411.76
3809    1563411.76
511     1563411.76
3798    1563411.76
625     1563411.76
3303    1563411.76
4095    1563411.76
1283    1563411.76
4209    1563411.76
1696    1563411.76
3511    1563411.76
816     1563411.76
245     1563411.76
1383    1563411.76
2071    1563411.76
3492    1563411.76
378     1563411.76
2971    1563411.76
2366    1563411.76
4414    1563411.76
2790    1563411.76
3979    1563411.76
193     1563411.76
1189    1563411.76
3516    1563411.76
810     1563411.76
4443    1563411.76
3697    1563411.76
235     1563411.76
1382    1563411.76
4384     540000.00
3418     115636.36
4396     834800.00
921      296444.44
3176     247166.66
650      550000.00
Name: fc99f9426, dtype: float64 1757    1563411.76
3809    1563411.76
511     1563411.76
3798    1563411.76
625     1563411.76
3303    1563411.76
4095    1563411.76
1283    1563411.76
4209    1563411.76
1696    1563411.76
351

Leading col found: e8a3423d6
Leading col found: 2135fa05a
[ 6 23 29]
6
difs 0 2
idx_value0 1757    1470000.0
3809    1470000.0
511     1470000.0
3798    1470000.0
625     1470000.0
3303    1470000.0
4095    1470000.0
1283    1470000.0
4209    1470000.0
1696    1470000.0
3511    1470000.0
816     1470000.0
245     1470000.0
1383    1470000.0
2071    1470000.0
3492    1470000.0
378     1470000.0
2971    1470000.0
2366    1470000.0
4414    1470000.0
2790    1470000.0
3979    1470000.0
193     1470000.0
1189    1470000.0
3516    1470000.0
810     1470000.0
4443    1470000.0
3697    1470000.0
235     1470000.0
1382    1470000.0
4384          0.0
3418          0.0
4396     480000.0
921           0.0
3176          0.0
650           0.0
Name: bf59c51c3, dtype: float64 1757    1470000.0
3809    1470000.0
511     1470000.0
3798    1470000.0
625     1470000.0
3303    1470000.0
4095    1470000.0
1283    1470000.0
4209    1470000.0
1696    1470000.0
3511    1470000.0
816     1470000.0
245     14700

Leading col found: 8e4d0fe45
Leading col found: c928b4b74
[ 6 23 29]
6
difs 4 1
idx_value0 1757    1033000.0
3809    1033000.0
511     1033000.0
3798    1033000.0
625     1033000.0
3303    1033000.0
4095    1033000.0
1283    1033000.0
4209    1033000.0
1696    1033000.0
3511    1033000.0
816     1033000.0
245     1033000.0
1383    1033000.0
2071    1033000.0
3492    1033000.0
378     1033000.0
2971    1033000.0
2366    1033000.0
4414    1033000.0
2790    1033000.0
3979    1033000.0
193     1033000.0
1189    1033000.0
3516    1033000.0
810     1033000.0
4443    1033000.0
3697    1033000.0
235     1033000.0
1382    1033000.0
4384     676000.0
3418      97000.0
4396          0.0
921      724000.0
3176          0.0
650           0.0
Name: d4029c010, dtype: float64 1757    1033000.0
3809    1033000.0
511     1033000.0
3798    1033000.0
625     1033000.0
3303    1033000.0
4095    1033000.0
1283    1033000.0
4209    1033000.0
1696    1033000.0
3511    1033000.0
816     1033000.0
245     10330

[['53aa182a2',
  '4e92107c6',
  '295408598',
  'b76bf3f19',
  '3305c8063',
  'd3a116347',
  'ac5260727',
  '199caef5d',
  '97ea72529',
  '1d4d5cd4a',
  '8fc7efaf0',
  '225fa9d61',
  '94f3dcaee',
  '4634c8fae',
  '052f633c1',
  '660fdbc58',
  '657dec16b',
  '7fa5bc19f',
  '7207afb67',
  'cda277b2a',
  'e9a473fbb',
  '3eac9a76e',
  '1c554649c',
  '86ffb104c',
  'b14d5014b',
  '8348ea8d3',
  'e3a4596f9',
  '49db469f5',
  'f928893ca',
  'aa610feec',
  'fa2a340da',
  '652142369',
  '947c7c3e8',
  'f81908ca5',
  '8160230fd',
  'c2d200f0e',
  'c99902a93',
  'd3a6362c5',
  '3ee95e3ef',
  '7f8027faf'],
 ['81de0d45e',
  '18562fc62',
  '543c24e33',
  '0256b6714',
  'd6006ff44',
  '6a323434b',
  'e3a38370e',
  '7c444370b',
  '8d2d050a2',
  '9657e51e1',
  '13f3a3d19',
  'b5c839236',
  '70f3033c6',
  'f4b374613',
  '16b532cdc',
  '849125d91',
  '88219c257',
  '74fb8f14c',
  'fd1102929',
  '699712087',
  '22501b58e',
  '9e9274b24',
  '2c42b0dce',
  '2c95e6e31',
  '5263c204d',
  '526ed2bec',
  '01f7de

In [74]:
len(extra_feats)

9

In [199]:

cols_2 = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11','73687e512',
                'aa164b93b','342e7eb03','cd24eae8a','8f3740670','2b2a10857','a00adf70e',
                '3a48a2cd2','a396ceeb9','9280f3d04','fec5eaf1a','5b943716b','22ed6dba3',
                '5547d6e11','e222309b0','5d3b81ef8','1184df5c2','2288333b4','f39074b55',
                'a8b721722','13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
                'b85fa8b27','2155f5e16']
# cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
#         'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
#         '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
#         'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
#         '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
#         '6619d81fc', '1db387535', 
#         'fc99f9426',
#         '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
#        ]
# x = extra_feats

In [240]:
# extra_feats[16]
# extra_feats = [extra_feats[11]]

In [229]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
for i, c in enumerate(extra_feats):
    if not i:
        x = pd.concat([train.loc[rows, ['target']+cols], train.loc[rows, c]], axis=1)
    else:
        x = pd.concat([x, train.loc[rows, c]], axis=1)
# display(train.loc[rows, extra_feats[25]])
display(x)

target    f190486d6    58e2e02e6    eeb9cd3aa    9fd594eec  \
1757    115636.36   1015000.00   1563411.76   1563411.76   1563411.76   
3809    834800.00    540000.00   1015000.00   1563411.76   1563411.76   
511     296444.44    115636.36    540000.00   1015000.00   1563411.76   
3798    247166.66    834800.00    115636.36    540000.00   1015000.00   
625     550000.00    296444.44    834800.00    115636.36    540000.00   
3303   3076666.66    247166.66    296444.44    834800.00    115636.36   
4095    440000.00    550000.00    247166.66    296444.44    834800.00   
1283   1600000.00    440000.00   3076666.66    550000.00    247166.66   
4209    466461.54         0.00    440000.00   3076666.66    550000.00   
1696   3147200.00   1600000.00         0.00    440000.00   3076666.66   
3511     75000.00    466461.54   1600000.00         0.00    440000.00   
816    1586888.88   3147200.00    466461.54   1600000.00         0.00   
245    1477600.00     75000.00   3147200.00    466461.54   1600000.00   
1383    310000.00   1477600.00   1586888.88     75000.00   3147200.00   
2071   3513333.34         0.00   1477600.00   1586888.88     75000.00   
3492    160000.00    310000.00         0.00   1477600.00   1586888.88   
378    2352551.72   3513333.34    310000.00         0.00   1477600.00   
2971    280000.00    160000.00   3513333.34    310000.00         0.00   
2366   5450500.00   2352551.72    160000.00   3513333.34    310000.00   
4414   1359000.00    280000.00   2352551.72    160000.00   3513333.34   
2790     60000.00   5450500.00    280000.00   2352551.72    160000.00   
3979  12000000.00   1359000.00   5450500.00    280000.00   2352551.72   
193     500000.00     60000.00   1359000.00   5450500.00    280000.00   
1189   1878571.42  12000000.00     60000.00   1359000.00   5450500.00   
3516    814800.00    500000.00  12000000.00     60000.00   1359000.00   
810     307000.00   1878571.42    500000.00  12000000.00     60000.00   
4443    528666.66    814800.00   1878571.42    500000.00  12000000.00   
3697    609200.00    307000.00    814800.00   1878571.42    500000.00   
235     406000.00    528666.66    307000.00    814800.00   1878571.42   
1382    448666.66    609200.00    528666.66    307000.00    814800.00   
4384    400000.00    448666.66    406000.00    609200.00    528666.66   
3418     60666.66         0.00    448666.66    406000.00    609200.00   
4396    267428.58    400000.00         0.00    448666.66    406000.00   
921    1304800.00     60666.66    400000.00         0.00    448666.66   
3176   8120000.00    267428.58     60666.66    400000.00         0.00   
650     380000.00   1304800.00    267428.58     60666.66    400000.00   

        6eef030c1    15ace8c9f    fb0f5dbfe   58e056e12    20aa07010  \
1757   1563411.76   1563411.76   1563411.76  1563411.76   1563411.76   
3809   1563411.76   1563411.76   1563411.76  1563411.76   1563411.76   
511    1563411.76   1563411.76   1563411.76  1563411.76   1563411.76   
3798   1563411.76   1563411.76   1563411.76  1563411.76   1563411.76   
625    1015000.00   1563411.76   1563411.76  1563411.76   1563411.76   
3303    540000.00   1015000.00   1563411.76  1563411.76   1563411.76   
4095    115636.36    540000.00   1015000.00  1563411.76   1563411.76   
1283    296444.44    834800.00    115636.36   540000.00   1015000.00   
4209    247166.66    296444.44    834800.00   115636.36    540000.00   
1696    550000.00    247166.66    296444.44   834800.00    115636.36   
3511   3076666.66    550000.00    247166.66   296444.44    834800.00   
816     440000.00   3076666.66    550000.00   247166.66    296444.44   
245          0.00    440000.00   3076666.66   550000.00    247166.66   
1383    466461.54   1600000.00         0.00   440000.00   3076666.66   
2071   3147200.00    466461.54   1600000.00        0.00    440000.00   
3492     75000.00   3147200.00    466461.54  1600000.00         0.00   
378    1586888.88     75000.00   3147200.00   466461.54   1600000.00   
2971   147760

# Find leaks

In [6]:
# from: https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2
def _get_leak(df, cols, extra_feats=[[]], lag=0):
#     f1 = list(set(f1))
#     f2 = list(set(f2))
    
    f1 = cols[:-lag-2]
    f2 = cols[lag+2:]
    for ef in extra_feats:
        f1 += ef[:-lag-2]
        f2 += ef[lag+2:]
    
    d1 = df[f1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[f2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [7]:
def compiled_leak_result():
    
    max_nlags = len(cols) - 2
    train_leak = train
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        train_leak[c] = _get_leak(train_leak, cols, extra_feats, i)
        
        leaky_cols.append(c)
#         train_leak = train.join(train_leak[leaky_cols+["compiled_leak", "nonzero_mean", "ID"]], on='ID', how='left')
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = train_leak["nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
        gc.collect()
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

In [241]:
train_leak, result = compiled_leak_result()

Processing lag 0
Leak values found in train 1373
% of correct leaks values in train  0.9970866715222141
Score (filled with nonzero mean) 1.5138333391635181
Processing lag 1
Leak values found in train 1974
% of correct leaks values in train  0.9979736575481256
Score (filled with nonzero mean) 1.2879157440143256
Processing lag 2
Leak values found in train 2363
% of correct leaks values in train  0.9978840457046128
Score (filled with nonzero mean) 1.1669163053029474
Processing lag 3
Leak values found in train 2606
% of correct leaks values in train  0.9980813507290868
Score (filled with nonzero mean) 1.0725450796345033
Processing lag 4
Leak values found in train 2774
% of correct leaks values in train  0.998197548666186
Score (filled with nonzero mean) 1.019695030587641
Processing lag 5
Leak values found in train 2927
% of correct leaks values in train  0.9969251793645371
Score (filled with nonzero mean) 0.9800751170415597
Processing lag 6
Leak values found in train 3049
% of correct leak

In [ ]:
result = pd.DataFrame.from_dict(result, orient='columns')
result.T

In [ ]:
result.to_csv('train_leaky_stat.csv', index=False)

In [ ]:
train_leak.head()

In [ ]:
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

In [ ]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [ ]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]
train_leak = rewrite_compiled_leak(train_leak, best_lag)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
train_res = train_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak.csv', index=False)

In [ ]:
def compiled_leak_result_test(max_nlags):
    test_leak = test[["ID", "target"] + cols]
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        test_leak[c] = _get_leak(test_leak, cols, i)
        
        leaky_cols.append(c)
        test_leak = test.join(
            test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [ ]:
test_leak, test_result = compiled_leak_result_test(max_nlags=38)

In [ ]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

In [ ]:
test_result.to_csv('test_leaky_stat.csv', index=False)

In [ ]:
test_leak = rewrite_compiled_leak(test_leak, best_lag)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
test_res = test_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak.csv', index=False)

In [ ]:
# test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = 0

In [ ]:
#submission
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

In [ ]:
sub.head()